<a href="https://colab.research.google.com/github/elhaddad2/NLP-Projet/blob/main/NLP_Projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PRE-PROCECING**

---



In [ ]:
!pip install textblob_fr 

     |████████████████████████████████| 561 kB 5.5 MB/s 


In [ ]:
import pandas as pd 
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import re
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
import string
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
data = pd.read_csv('/content/Comments.csv')
data.head()

,Unnamed: 0,Name,Comment,Month,Year
0,0,Réassurez-moi,Ravi ! La recherche et la comparaison des mutu...,8,2019
1,1,Réassurez-moi,Très satisfaite ! Un conseiller à l'écoute et ...,8,2019
2,2,Réassurez-moi,Je vous avais contacté pour la mise en place e...,8,2019
3,3,Réassurez-moi,Quelques mots sur Réassurez-moi. Je suis très ...,8,2019
4,4,Réassurez-moi,Votre cabinet m'a permis de pouvoir bénéficier...,8,2019


In [ ]:
data= data.drop(['Unnamed: 0'],axis=1)

In [ ]:
NAN = [(c, data[c].isna().mean()*100) for c in data]
NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])
NAN.sort_values("percentage", ascending=False)

,column_name,percentage
1,Comment,0.14523
0,Name,0.00000
2,Month,0.00000
3,Year,0.00000


In [ ]:
data =data.dropna()

In [ ]:
NAN = [(c, data[c].isna().mean()*100) for c in data]
NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])
NAN.sort_values("percentage", ascending=False)

,column_name,percentage
0,Name,0.0
1,Comment,0.0
2,Month,0.0
3,Year,0.0


In [ ]:
data["Comment"]= data["Comment"].str.lower()

In [ ]:
AComment=[]
for comment in data["Comment"].apply(str):
    Word_Tok = []
    for word in  re.sub("\W"," ",comment ).split():
        Word_Tok.append(word)
    AComment.append(Word_Tok)

In [ ]:
data["Word_Tok"]= AComment
data.head()

,Name,Comment,Month,Year,Word_Tok
0,Réassurez-moi,ravi ! la recherche et la comparaison des mutu...,8,2019,"[ravi, la, recherche, et, la, comparaison, des..."
1,Réassurez-moi,très satisfaite ! un conseiller à l'écoute et ...,8,2019,"[très, satisfaite, un, conseiller, à, l, écout..."
2,Réassurez-moi,je vous avais contacté pour la mise en place e...,8,2019,"[je, vous, avais, contacté, pour, la, mise, en..."
3,Réassurez-moi,quelques mots sur réassurez-moi. je suis très ...,8,2019,"[quelques, mots, sur, réassurez, moi, je, suis..."
4,Réassurez-moi,votre cabinet m'a permis de pouvoir bénéficier...,8,2019,"[votre, cabinet, m, a, permis, de, pouvoir, bé..."


In [ ]:
stop_words=set(STOP_WORDS)

deselect_stop_words = ['n\'', 'ne','pas','plus','personne','aucun','ni','aucune','rien']
for w in deselect_stop_words:
    if w in stop_words:
        stop_words.remove(w)
    else:
        continue

In [ ]:
AllfilteredComment=[]
for comment in data["Word_Tok"]:
    filteredComment = [w for w in comment if not ((w in stop_words) or (len(w) == 1))]
    AllfilteredComment.append(' '.join(filteredComment))

In [ ]:
data["CommentAferPreproc"]=AllfilteredComment
data.head()

,Name,Comment,Month,Year,Word_Tok,CommentAferPreproc
0,Réassurez-moi,ravi ! la recherche et la comparaison des mutu...,8,2019,"[ravi, la, recherche, et, la, comparaison, des...",ravi recherche comparaison mutuelles simples r...
1,Réassurez-moi,très satisfaite ! un conseiller à l'écoute et ...,8,2019,"[très, satisfaite, un, conseiller, à, l, écout...",satisfaite conseiller écoute sympathique bon p...
2,Réassurez-moi,je vous avais contacté pour la mise en place e...,8,2019,"[je, vous, avais, contacté, pour, la, mise, en...",contacté mise place janvier contrat mutuelle s...
3,Réassurez-moi,quelques mots sur réassurez-moi. je suis très ...,8,2019,"[quelques, mots, sur, réassurez, moi, je, suis...",mots réassurez satisfait services cabinet trou...
4,Réassurez-moi,votre cabinet m'a permis de pouvoir bénéficier...,8,2019,"[votre, cabinet, m, a, permis, de, pouvoir, bé...",cabinet permis pouvoir bénéficier contrat comp...


In [ ]:
senti_list = []
Labels = []
for i in data["CommentAferPreproc"]:
    vs = tb(i).sentiment[0]
    if (vs > 0):
        senti_list.append('Positive')
        Labels.append(1)
    elif (vs < 0):
        senti_list.append('Negative')
        Labels.append(-1)
    else:
        senti_list.append('Neutral') 
        Labels.append(0)

In [ ]:
data["sentiment"]=senti_list
data.head()


,Name,Comment,Month,Year,Word_Tok,CommentAferPreproc,sentiment
0,Réassurez-moi,ravi ! la recherche et la comparaison des mutu...,8,2019,"[ravi, la, recherche, et, la, comparaison, des...",ravi recherche comparaison mutuelles simples r...,Positive
1,Réassurez-moi,très satisfaite ! un conseiller à l'écoute et ...,8,2019,"[très, satisfaite, un, conseiller, à, l, écout...",satisfaite conseiller écoute sympathique bon p...,Positive
2,Réassurez-moi,je vous avais contacté pour la mise en place e...,8,2019,"[je, vous, avais, contacté, pour, la, mise, en...",contacté mise place janvier contrat mutuelle s...,Positive
3,Réassurez-moi,quelques mots sur réassurez-moi. je suis très ...,8,2019,"[quelques, mots, sur, réassurez, moi, je, suis...",mots réassurez satisfait services cabinet trou...,Positive
4,Réassurez-moi,votre cabinet m'a permis de pouvoir bénéficier...,8,2019,"[votre, cabinet, m, a, permis, de, pouvoir, bé...",cabinet permis pouvoir bénéficier contrat comp...,Positive


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

vectorizer = TfidfVectorizer (max_features=2500, min_df=0.2, max_df=0.8, stop_words=stopwords.words('french'))
processed_features = vectorizer.fit_transform(data["CommentAferPreproc"]).toarray()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
processed_features

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 0.],
       [0., 1., 0.]])

Built Machine learning Module (RandomForest)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(processed_features, Labels, test_size=0.2, random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
text_classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
predictions = text_classifier.predict(X_test)
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[  17    0  710]
 [   2    0  313]
 [  18    0 1141]]
              precision    recall  f1-score   support

          -1       0.46      0.02      0.04       727
           0       0.00      0.00      0.00       315
           1       0.53      0.98      0.69      1159

    accuracy                           0.53      2201
   macro avg       0.33      0.34      0.24      2201
weighted avg       0.43      0.53      0.38      2201

0.526124488868696


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
Number_sentiment= data.groupby(["sentiment"])["Name"].count().reset_index().reset_index(drop=True)

In [ ]:
fig = px.histogram(data, x="sentiment",color="sentiment")
fig.update_layout(
    title_text='Sentiment of reviews', # title of plot
    xaxis_title_text='Sentiment', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.2, 
    bargroupgap=0.1
)
fig.show()

In [ ]:
fig = px.pie(Number_sentiment, values=Number_sentiment['Name'], names=Number_sentiment['sentiment'], color_discrete_sequence=px.colors.sequential.Emrld
)
fig.show()

In [ ]:
fig = px.histogram(data, x="Name",color="Name")
fig.update_layout(
    title_text='Number of Comments per Assurance', # title of plot
    xaxis_title_text='Assurance', # xaxis label
    yaxis_title_text='Number of Comments', # yaxis label
    bargap=0.2, 
    bargroupgap=0.1
)
fig.show()

In [ ]:
fig = px.histogram(data, x="Year",color="Year")
fig.update_layout(
    title_text='Number of Comments per Year', # title of plot
    xaxis_title_text='Year', # xaxis label
    yaxis_title_text='Number of Comments', # yaxis label
    bargap=0.2, 
    bargroupgap=0.1
)
fig.show()